**why we use polynomial regression?**
1. if there is a non-linear relationship between features and labels,
2. interaction terms between faetures.

> thus polynomial regression has a better performance than Linear Regression(Because it conciders higher order relationship between features).
> **scikit learn** preprocessing library helps us to have higher order relationship between features(x1^2, X2^2,...) and intraction terms(x1x2, x1x2,...) by means of **PolynomialFeatures.**  

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
df=pd.read_csv('../input/real-estate-price-prediction/Real estate.csv')

**data overview(it helps us to have a better understanding of data set)**

In [ ]:
df.head()

In [ ]:
df.corr()

In [ ]:
df.info()

**EDA** (plots to visualize dataset)

In [ ]:
plt.figure(figsize=(8,3))
sns.displot(df['Y house price of unit area'], bins=30, kde=True)

plt.xlabel('Y house price of unit area')

In [ ]:
sns.kdeplot(df['X2 house age'])
sns.kdeplot(df['Y house price of unit area'])

plt.legend()

**determine the features and the labels:**

In [ ]:
X=df.drop('Y house price of unit area' , axis=1)
y=df['Y house price of unit area']

**preprocessing**

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

In [ ]:
polynomial_converter = PolynomialFeatures(degree=2, include_bias=False)

In [ ]:
poly_features = polynomial_converter.fit_transform(X)

In [ ]:
X.shape

In [ ]:
poly_features.shape

we saw how **PolynomialFeatures** expand the features

**split data to train and test**

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
 X_train, X_test, y_train, y_test = train_test_split( poly_features, y, test_size=0.3, random_state=101)

**train the model**

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
polymodel=LinearRegression()

In [ ]:
polymodel.fit(X_train,y_train)

**prediction:**

In [ ]:
y_pred=polymodel.predict(X_test)

for better underestanding we can have a dictionary as below

In [ ]:
pd.DataFrame({'Y_test':y_test,'Y_pred':y_pred, 'residual':(y_test-y_pred)})

**Evaluating model performance**

In [ ]:
from sklearn import metrics

In [ ]:
MAE=metrics.median_absolute_error(y_test,y_pred)
MSE=metrics.mean_squared_error(y_test,y_pred)
RMSE=np.sqrt(MSE)

In [ ]:
pd.DataFrame([MAE,MSE,RMSE],index=['MAE','MSE','RMSE'],columns=['metrics'])

* the question is, since when we can add order or how can we choose the optimal degree for the polynimial?
Bias-Variance Trade off hepls us to understand it better.(over fitting & under fitting)
* a good model has lower error with growth of complexity(polynomial regression hyperparameter) on training and test set but we have to be carefull about overfitting.

**adjusting model hyperparameters**

In [ ]:
#Train List of RMSE per degree
train_RMSE_List=[]
#test list of RMSE per degree
test_RMSE_list=[]


for degree in range(1,10):
    #preprocessing
    #create poly dataset for degree d
    polynomial_convertor=PolynomialFeatures(degree=degree, include_bias=False)
    poly_features=polynomial_convertor.fit_transform(X)
    
    #split the data set
    X_train, X_test, y_train, y_test = train_test_split( poly_features, y, test_size=0.3, random_state=101)
    
    #train the model
    polymodel=LinearRegression()
    polymodel.fit(X_train,y_train)
     
    #prediction
    y_train_pred=polymodel.predict(X_train)
    y_test_pred=polymodel.predict(X_test)
    
    #evaluating
    
    #RMSE of train set
    train_RMSE=np.sqrt(metrics.mean_squared_error(y_train,y_train_pred))
    
    #RMSE of test set
    test_RMSE=np.sqrt(metrics.mean_squared_error(y_test,y_test_pred))
    
    #append the RMSE to the train and test list
    train_RMSE_List.append(train_RMSE)
    test_RMSE_list.append(test_RMSE)

In [ ]:
train_RMSE_List

In [ ]:
test_RMSE_list

so as we see degree=2 is the best answer 

In [ ]:
plt.plot(range(1,5),train_RMSE_List[:4],label='Train RMSE')
plt.plot(range(1,5),test_RMSE_list[:4],label='Test RMSE')
plt.xlabel('polynomial degree')
plt.ylabel('RMSE')
plt.legend()